In [29]:
import torch
import torch.nn.functional as F
from pytorch_pretrained_bert import BertTokenizer
from bertModel import BertClassification
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

In [16]:
labels = {0:'neutral', 1:'positive',2:'negative'}
num_labels= len(labels)
vocab = "finance-uncased"
vocab_path = 'analyst_tone/vocab'
pretrained_weights_path = "analyst_tone/pretrained_weights" # this is pre-trained FinBERT weights
fine_tuned_weight_path = "analyst_tone/fine_tuned.pth"      # this is fine-tuned FinBERT weights
max_seq_length=512
device='cpu'#'cuda:1'
tokenizer = BertTokenizer(vocab_file = vocab_path, do_lower_case = True, do_basic_tokenize = True)

In [21]:
model = BertClassification(weight_path= pretrained_weights_path, num_labels=num_labels, vocab=vocab)
model.load_state_dict(torch.load(fine_tuned_weight_path, map_location=torch.device('cpu')))
model.to(device);
model.eval();

In [26]:
df = pd.read_csv('../tweets_influencers.csv')
df['sentiment'] = ''
df.head()

,id,user,fullname,url,timestamp,replies,likes,retweets,text,sentiment
0,1.220894e+16,gavinandresen,Gavin Andresen,NaN,2010-12-07 18:16:53+00:00,0.0,0.0,4.0,Launched my second #bitcoin project today: htt...,
1,7.123706e+16,gavinandresen,Gavin Andresen,NaN,2011-05-19 13:33:33+00:00,0.0,5.0,6.0,"Yesterday it was #bitcoin in Slate, today Gizm...",
2,2.035533e+17,ErikVoorhees,Erik Voorhees,NaN,2012-05-18 16:30:57+00:00,0.0,5.0,9.0,@zerohedge Bitcoin wasn't hacked (and has neve...,
3,2.078903e+17,rogerkver,Roger Ver,NaN,2012-05-30 17:44:50+00:00,4.0,3.0,3.0,Any suggestions for which bitcoin logo to put ...,
4,2.224597e+17,ErikVoorhees,Erik Voorhees,NaN,2012-07-09 22:38:14+00:00,76.0,1382.0,362.0,#Bitcoin just broke $7.00... now up 40% in the...,


In [37]:
for index, row in tqdm(df.iterrows(), total=len(df)):
    sent = row['text']
    tokenized_sent = tokenizer.tokenize(sent)
    if len(tokenized_sent) > max_seq_length:
        tokenized_sent = tokenized_sent[:max_seq_length]
    
    ids_review  = tokenizer.convert_tokens_to_ids(tokenized_sent)
    mask_input = [1]*len(ids_review)        
    padding = [0] * (max_seq_length - len(ids_review))
    ids_review += padding
    mask_input += padding
    input_type = [0]*max_seq_length
    
    input_ids = torch.tensor(ids_review).to(device).reshape(-1, max_seq_length)
    attention_mask =  torch.tensor(mask_input).to(device).reshape(-1, max_seq_length)
    token_type_ids = torch.tensor(input_type).to(device).reshape(-1, max_seq_length)
    
    with torch.set_grad_enabled(False):
        outputs = model(input_ids, token_type_ids, attention_mask)
        outputs = F.softmax(outputs,dim=1)
        print(sent, '\nFinBERT predicted sentiment: ', labels[torch.argmax(outputs).item()], '\n')
        df.loc[index, 'sentiment'] = labels[torch.argmax(outputs).item()]
        
    if index > 100:
        break

Launched my second #bitcoin project today: https://clearcoin.appspot.com/ 
FinBERT predicted sentiment:  neutral 

Yesterday it was #bitcoin in Slate, today Gizmodo... it is a PR avalanche!  Maybe I picked the wrong week to go to France. 
FinBERT predicted sentiment:  neutral 

@zerohedge Bitcoin wasn't hacked (and has never been), it was an exchange website (Bitcoinica) which got hacked. 
FinBERT predicted sentiment:  neutral 

Any suggestions for which bitcoin logo to put on the back of my new jujitsu gi? #bjj #jujitsu #munials #Bitcoin http://t.co/uaTZjeUa 
FinBERT predicted sentiment:  neutral 

#Bitcoin just broke $7.00... now up 40% in the last 30 days and most people don't even know what we're up to ;) #forex #finance #gold #money 
FinBERT predicted sentiment:  positive 

Just discovered that my Bitcoin casino game, SatoshiDice, is now responsible for over 50% of all Bitcoin transactions ever (over 1.37m txs) 
FinBERT predicted sentiment:  neutral 

I just heard that @coinbase j

BREAKING: Janet Yellen receives enough votes in Senate to be confirmed as person to be most humiliated by Bitcoin in 2014 
FinBERT predicted sentiment:  neutral 

If you're a small country that would like to be the richest per capita, buy up all the Bitcoin you can and announce adoption. You're welcome 
FinBERT predicted sentiment:  neutral 

If Bitcoin, now at 5M users, follows Facebook's growth, we should reach 500 million users by 2019, and 1 billion Bitcoin users by 2021. 
FinBERT predicted sentiment:  positive 

Ethereum is probably the first programming language I will teach myself - who wouldn't want the ability to program smart BTC contracts? 
FinBERT predicted sentiment:  neutral 

The Internet allows any two individuals to transfer data without permission from any central authority. Bitcoin does the same for value. 
FinBERT predicted sentiment:  neutral 

Fed chair Janet Yellen:  "the fed doesn't have authority to supervise or regulate Bitcoin" http://t.co/2ItslicxoX 
FinBERT

In [39]:
df.head(100)

,id,user,fullname,url,timestamp,replies,likes,retweets,text,sentiment
0,1.220894e+16,gavinandresen,Gavin Andresen,NaN,2010-12-07 18:16:53+00:00,0.0,0.0,4.0,Launched my second #bitcoin project today: htt...,neutral
1,7.123706e+16,gavinandresen,Gavin Andresen,NaN,2011-05-19 13:33:33+00:00,0.0,5.0,6.0,"Yesterday it was #bitcoin in Slate, today Gizm...",neutral
2,2.035533e+17,ErikVoorhees,Erik Voorhees,NaN,2012-05-18 16:30:57+00:00,0.0,5.0,9.0,@zerohedge Bitcoin wasn't hacked (and has neve...,neutral
3,2.078903e+17,rogerkver,Roger Ver,NaN,2012-05-30 17:44:50+00:00,4.0,3.0,3.0,Any suggestions for which bitcoin logo to put ...,neutral
4,2.224597e+17,ErikVoorhees,Erik Voorhees,NaN,2012-07-09 22:38:14+00:00,76.0,1382.0,362.0,#Bitcoin just broke $7.00... now up 40% in the...,positive
...,...,...,...,...,...,...,...,...,...,...
95,5.646002e+17,gavinandresen,Gavin Andresen,NaN,2015-02-09 01:42:37+00:00,7.0,22.0,45.0,Robin Hanson on #bitcoin ; my only quibble is ...,neutral
96,5.655334e+17,ToneVays,Tone Vays [RIP Tyler Jenks],NaN,2015-02-11 15:30:55+00:00,2.0,2.0,1.0,Stuck at $223 (fib) again but #bitcoin is coil...,neutral
97,5.676711e+17,notsofast,notsofast,NaN,2015-02-17 13:05:05+00:00,0.0,2.0,1.0,"""Smart Contracts..."" http://t.co/qjtctVLmBT vi...",neutral
98,5.731372e+17,ChrisDunnTV,Chris Dunn,NaN,2015-03-04 15:05:21+00:00,10.0,24.0,17.0,"Nice bull trend in #bitcoin, watching $300's h...",neutral
